## Data pre-processing

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/interim/nhanes_data_interim.csv")

### Convert categorical variables represented by float to int

In [3]:
df["diabetes"] = df["diabetes"].astype(int)
df["current_smoking_freq"] = df["current_smoking_freq"].astype(int)

### Transform highly-skewed features

In [4]:
# Log transformation on right-skewed features

cols_to_transform = ["hba1c_percentage", "fpg_mmol_L", "hdl_mmol_L"]

for col in cols_to_transform:
    df[col + "_log"] = np.log(df[col])

# Drop original columns

df.drop(cols_to_transform, axis=1)

,bmi,high_blood_pressure,gender,age,diabetes,lifetime_100_cigs_smoked,current_smoking_freq,hba1c_percentage_log,fpg_mmol_L_log,hdl_mmol_L_log
0,27.0,1,1,43,2,1,3,1.722767,1.835776,0.148420
1,33.5,1,1,66,2,1,3,1.722767,1.704748,0.438255
2,30.2,0,1,34,2,1,3,1.629241,1.713798,0.173953
3,27.3,0,0,56,2,1,3,1.609438,1.752672,0.565314
4,21.4,1,1,80,2,1,3,1.526056,1.597365,0.751416
...,...,...,...,...,...,...,...,...,...,...
1204,30.2,0,0,51,2,1,1,1.774952,1.937302,0.277632
1205,35.7,0,1,33,1,1,3,2.251292,2.424803,-0.162519
1206,32.6,0,0,46,2,1,3,1.629241,1.662030,0.350657
1207,30.5,1,0,80,2,1,3,1.722767,1.809927,0.215111


### Identify outliers

In [5]:
def three_sd_range(series):
    mean = series.mean()
    sd = series.std()
    lower_bound = mean - 3 * sd
    upper_bound = mean + 3 * sd
    
    return (lower_bound, upper_bound)

In [6]:
for col_name in ["bmi", "age", "hba1c_percentage_log", "fpg_mmol_L_log", "hdl_mmol_L_log"]:
    lower_bound, upper_bound = three_sd_range(df[col_name])
    outlier_present = (df[col_name].min() < lower_bound or df[col_name].max() > upper_bound)
    print(f"{col_name} has at least one outlier: {outlier_present}")
    
    if outlier_present:
        curr_length = df[col_name].count()
        df = df[(df[col_name] >= lower_bound) & (df[col_name] <= upper_bound)]
        new_length = df[col_name].count()
        print(f"{curr_length - new_length} row has been removed")
    
    print()

bmi has at least one outlier: True
11 row has been removed

age has at least one outlier: False

hba1c_percentage_log has at least one outlier: True
31 row has been removed

fpg_mmol_L_log has at least one outlier: True
27 row has been removed

hdl_mmol_L_log has at least one outlier: True
6 row has been removed



In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1134 entries, 0 to 1208
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bmi                       1134 non-null   float64
 1   high_blood_pressure       1134 non-null   int64  
 2   gender                    1134 non-null   int64  
 3   age                       1134 non-null   int64  
 4   diabetes                  1134 non-null   int32  
 5   hba1c_percentage          1134 non-null   float64
 6   fpg_mmol_L                1134 non-null   float64
 7   hdl_mmol_L                1134 non-null   float64
 8   lifetime_100_cigs_smoked  1134 non-null   int64  
 9   current_smoking_freq      1134 non-null   int32  
 10  hba1c_percentage_log      1134 non-null   float64
 11  fpg_mmol_L_log            1134 non-null   float64
 12  hdl_mmol_L_log            1134 non-null   float64
dtypes: float64(7), int32(2), int64(4)
memory usage: 115.2 KB


In [8]:
# Save the pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed.csv", index=False)

### Categorical encoding

In [9]:
# Map multiclass values to more readable strings to prepare for one-hot encoding
df["diabetes"] = df["diabetes"].map({1: "yes", 2: "no", 3: "borderline"})
df["current_smoking_freq"] = df["current_smoking_freq"].map({1: "every_day", 2: "some_days", 3: "none"})

In [10]:
# One-hot encode the multiclass columns "diabetes" and "current_smoking_frequency"
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, dtype=int)
one_hot_encoded = encoder.fit_transform(df[["diabetes", "current_smoking_freq"]])

encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(["diabetes", "current_smoking_freq"]))

# Concatenate the original DataFrame with the one-hot encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original multiclass columns after encoding
df = df.drop(columns=["diabetes", "current_smoking_freq"], axis=1)

In [11]:
# Resultant DataFrame after one-hot encoding
df.sample(5)

,bmi,high_blood_pressure,gender,age,hba1c_percentage,fpg_mmol_L,hdl_mmol_L,lifetime_100_cigs_smoked,hba1c_percentage_log,fpg_mmol_L_log,hdl_mmol_L_log,diabetes_borderline,diabetes_no,diabetes_yes,current_smoking_freq_every_day,current_smoking_freq_none,current_smoking_freq_some_days
823,21.7,0.0,1.0,40.0,5.0,5.27,1.29,1.0,1.609438,1.662030,0.254642,0.0,1.0,0.0,0.0,1.0,0.0
460,19.5,0.0,1.0,22.0,5.4,5.11,1.76,1.0,1.686399,1.631199,0.565314,0.0,1.0,0.0,0.0,1.0,0.0
371,22.5,0.0,1.0,67.0,5.0,5.38,1.29,1.0,1.609438,1.682688,0.254642,0.0,1.0,0.0,1.0,0.0,0.0
463,31.2,0.0,1.0,34.0,5.6,5.66,1.06,1.0,1.722767,1.733424,0.058269,0.0,1.0,0.0,0.0,1.0,0.0
602,36.4,0.0,1.0,80.0,5.7,6.05,1.40,1.0,1.740466,1.800058,0.336472,0.0,1.0,0.0,0.0,1.0,0.0


In [12]:
# Save the one-hot encoded pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed_one_hot.csv", index=False)